# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: gpt2
* Evaluation approach: 
* Fine-tuning dataset: https://huggingface.co/datasets/dair-ai/emotion

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from datasets import load_dataset

ds = load_dataset("dair-ai/emotion", "split")

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Let's use a lambda function to tokenize all the examples
splits = ["train", "test"]
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = ds[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset["train"]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 16000
})

In [3]:
train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

print(train_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 16000
})
Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 2000
})


In [4]:
train_dataset[0]

{'text': 'i didnt feel humiliated',
 'label': 0,
 'input_ids': [72, 42547, 1254, 42659],
 'attention_mask': [1, 1, 1, 1]}

In [5]:
test_dataset[0]

{'text': 'im feeling rather rotten so im not very ambitious right now',
 'label': 0,
 'input_ids': [320, 4203, 2138, 36371, 523, 545, 407, 845, 14742, 826, 783],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=6,
    id2label={0: "sadness", 1:"joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"},
    label2id = {"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5}
)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=6, bias=False)
)


In [8]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
import numpy as np
import torch

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Ensure GPT-2 has a padding token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Set pad_token explicitly
tokenizer.pad_token = tokenizer.pad_token or '[PAD]'

# Resize token embeddings to account for new special tokens
model.resize_token_embeddings(len(tokenizer))

# Set pad_token_id in model config
model.config.pad_token_id = tokenizer.pad_token_id

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./result",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt"),
    compute_metrics=compute_metrics,
)

trainer.evaluate()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 5.817080020904541,
 'eval_accuracy': 0.3445,
 'eval_runtime': 5.6411,
 'eval_samples_per_second': 354.54,
 'eval_steps_per_second': 22.159}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [9]:
from peft import LoraConfig
config = LoraConfig()

In [10]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [11]:
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,740,096 || trainable%: 0.2364211744714386


In [12]:
print(train_dataset.features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


In [13]:
print(tokenized_dataset["train"].column_names)

['text', 'label', 'input_ids', 'attention_mask']


In [ ]:
# tokenized_dataset["train"] = tokenized_dataset["train"].rename_column("label", "labels")
# tokenized_dataset["test"] = tokenized_dataset["test"].rename_column("label", "labels")

In [23]:
tokenized_dataset1 = {}
tokenized_dataset1["train"] = tokenized_dataset["train"].remove_columns(["text"])
tokenized_dataset1["test"] = tokenized_dataset["test"].remove_columns(["text"])
print(len(tokenized_dataset1["train"]), len(tokenized_dataset1["test"]))
tokenized_dataset1["train"] = tokenized_dataset1["train"].rename_column("label", "labels")
tokenized_dataset1["test"] = tokenized_dataset1["test"].rename_column("label", "labels")

16000 2000


In [24]:
fine_tuned_trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./result",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=3,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["test"],
    train_dataset=tokenized_dataset1["train"],
    eval_dataset=tokenized_dataset1["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt"),
    compute_metrics=compute_metrics,
)
fine_tuned_trainer.train()

IndexError: Invalid key: 14881 is out of bounds for size 0

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [ ]:
# Saving the model
model.save("/tmp/your_model_name")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.